Use Case 9
==========

Problem Definition:
-------------------

    A climate scientist wishes to analyse potential correlations between Aerosol and Cloud ECVs.
    
Required Toolbox Features:
--------------------------

    * Access to and ingestion of ESA CCI Aerosol and Cloud data (Aerosol Optical Depth and Cloud Fraction)
    * Geometric adjustments
    * Spatial (point, polygon) and temporal subsetting
    * Visualisation of both times series at the same time: e.g. time series plot, time series animation
    * Correlation analysis, scatter-plot of correlation statistics, saving of image and correlation statistics on disk (format options)

Ingest data and create datasets
-------------------------------

In [1]:
from ect.core.io import DATA_STORE_REGISTRY
from ect.core.monitor import ConsoleMonitor
import ect.ops

data_store = DATA_STORE_REGISTRY.get_data_store('default')
data_store

FileSetFileStore('/home/ccitbx/.ect/data_stores/esa_cci_portal_ftp')

In [2]:
data_source_aerosol = data_store.query('AEROSOL_AATSR_SU_L3_V4.21_MONTHLY')
data_source_clouds = data_store.query('CLOUD_L3C_AVHRR_NOAA-16_PHASE1_V1.0')

Retrieve data by synchronizing with the ftp server

In [ ]:
monitor = ConsoleMonitor()
data_source_aerosol[0].sync(('2007-01-01','2009-01-01'), monitor)

Synchronising AEROSOL_AATSR_SU_L3_V4.21_MONTHLY: started
Synchronising aerosol/data/AATSR_SU/L3/v4.21/MONTHLY/2007, contains 12 file(s), total size 70.9 MiB
Synchronising AEROSOL_AATSR_SU_L3_V4.21_MONTHLY:   0% aerosol/data/AATSR_SU/L3/v4.21/MONTHLY/2007
200701-ESACCI-L3C_AEROSOL-AER_PRODUCTS-AATSR_ENVISAT-SU_MONTHLY-v4.21.nc: local file is up-to-date
200702-ESACCI-L3C_AEROSOL-AER_PRODUCTS-AATSR_ENVISAT-SU_MONTHLY-v4.21.nc: local file is up-to-date
200703-ESACCI-L3C_AEROSOL-AER_PRODUCTS-AATSR_ENVISAT-SU_MONTHLY-v4.21.nc: local file is up-to-date
200704-ESACCI-L3C_AEROSOL-AER_PRODUCTS-AATSR_ENVISAT-SU_MONTHLY-v4.21.nc: local file is up-to-date
200705-ESACCI-L3C_AEROSOL-AER_PRODUCTS-AATSR_ENVISAT-SU_MONTHLY-v4.21.nc: local file is up-to-date
200706-ESACCI-L3C_AEROSOL-AER_PRODUCTS-AATSR_ENVISAT-SU_MONTHLY-v4.21.nc: local file is up-to-date
200707-ESACCI-L3C_AEROSOL-AER_PRODUCTS-AATSR_ENVISAT-SU_MONTHLY-v4.21.nc: local file is up-to-date
200708-ESACCI-L3C_AEROSOL-AER_PRODUCTS-AATSR_ENVISAT

In [ ]:
data_source_clouds[0].sync(('2007-01-01','2009-01-01'), monitor)

Synchronising CLOUD_L3C_AVHRR_NOAA-16_PHASE1_V1.0: started
Synchronising cloud/data/L3C/avhrr_noaa-16/phase1_v1.0/2007/01, contains 1 file(s), total size 160.2 MiB
Synchronising CLOUD_L3C_AVHRR_NOAA-16_PHASE1_V1.0:   0% cloud/data/L3C/avhrr_noaa-16/phase1_v1.0/2007/01
200701-ESACCI-L3C_CLOUD-CLD_PRODUCTS-AVHRR_NOAA-16-fv1.0.nc: local file is up-to-date
Synchronising CLOUD_L3C_AVHRR_NOAA-16_PHASE1_V1.0:   4% cloud/data/L3C/avhrr_noaa-16/phase1_v1.0/2007/01
Synchronising cloud/data/L3C/avhrr_noaa-16/phase1_v1.0/2007/02, contains 1 file(s), total size 160.2 MiB
Synchronising CLOUD_L3C_AVHRR_NOAA-16_PHASE1_V1.0:   4% cloud/data/L3C/avhrr_noaa-16/phase1_v1.0/2007/02
200702-ESACCI-L3C_CLOUD-CLD_PRODUCTS-AVHRR_NOAA-16-fv1.0.nc: local file is up-to-date
Synchronising CLOUD_L3C_AVHRR_NOAA-16_PHASE1_V1.0:   8% cloud/data/L3C/avhrr_noaa-16/phase1_v1.0/2007/02
Synchronising cloud/data/L3C/avhrr_noaa-16/phase1_v1.0/2007/03, contains 1 file(s), total size 160.2 MiB
Synchronising CLOUD_L3C_AVHRR_NOAA

In [ ]:
time_range = tuple(['2008-01-01','2009-01-01'])
dataset_aerosol = data_source_aerosol[0].open_dataset(time_range)
dataset_clouds = data_source_clouds[0].open_dataset(time_range)
print(dataset_clouds._wrapped_dataset.dims)
print(dataset_aerosol._wrapped_dataset.dims)

Filter datasets to select the desired variables
-----------------------------------------------

In [ ]:
clouds_filter = tuple(['cc_total'])
aerosol_filter = tuple(['AOD550_mean'])

dataset_clouds = dataset_clouds.filter(clouds_filter)
dataset_aerosol = dataset_aerosol.filter(aerosol_filter)
print(dataset_clouds._wrapped_dataset.dims)
print(dataset_aerosol._wrapped_dataset.dims)

In [ ]:
ect.ops.harmonize([dataset_clouds, dataset_aerosol])

In [ ]:
print(dataset_clouds._wrapped_dataset.dims)
print(dataset_aerosol._wrapped_dataset.dims)

Co-register datasets by resampling
----------------------------------

In [ ]:
dataset_aerosol = ect.ops.coregister(dataset_clouds, dataset_aerosol, 'bilinear')

In [ ]:
print(dataset_clouds._wrapped_dataset.dims)
print(dataset_aerosol._wrapped_dataset.dims)

Select the desired spatial region
---------------------------------

In [ ]:
dataset_aerosol = ect.ops.subset_spatial(dataset_aerosol, lat=[40.,60.], lon=[40.,60.])
dataset_clouds = ect.ops.subset_spatial(dataset_clouds, lat=[40.,60.], lon=[40.,60.])
print(dataset_clouds._wrapped_dataset.dims)
print(dataset_aerosol._wrapped_dataset.dims)

Select the desired temporal region
----------------------------------

In [ ]:
dataset_aerosol = ect.ops.subset_temporal_index(dataset_aerosol, time=[0,10])
dataset_clouds = ect.ops.subset_temporal_index(dataset_clouds, time=[0,10])
print(dataset_clouds._wrapped_dataset.dims)
print(dataset_aerosol._wrapped_dataset.dims)

Retrieve and plot timeseries
----------------------------

In [ ]:
timeseries_aerosol_point = ect.ops.timeseries(dataset_aerosol, lat=50., lon=50.)
timeseries_clouds_point = ect.ops.timeseries(dataset_clouds, lat=50., lon=50.)
timeseries_aerosol_mean = ect.ops.timeseries_mean(dataset_aerosol)
timeseries_clouds_mean = ect.ops.timeseries_mean(dataset_clouds)
print(timeseries_aerosol_mean.wrapped_dataset)
print(timeseries_clouds_mean.wrapped_dataset)

In [ ]:
len(timeseries_aerosol_mean.wrapped_dataset.data_vars)

Do correlation analysis
-----------------------

In [ ]:
correlation = ect.ops.pearson_correlation(timeseries_clouds_mean, timeseries_aerosol_mean)
print(correlation)

Create a scatter plot, save image
---------------------------------

In [ ]:
pass

Save correlation statistics
---------------------------

In [ ]:
pass